In [1]:
from os import listdir
from os.path import isfile, join
import string
import numpy as np
import pandas as pd
import math
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn import preprocessing
 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path="C:\\Users\\monac\\Downloads\\20_newsgroups\\20_newsgroups"
folders = [f for f in listdir(path)]

In [3]:
folders

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
categories = [ 'rec.sport.hockey','talk.politics.misc','sci.space','comp.graphics', 'sci.med']


In [5]:
files = []
for folder_name in categories:
    folder_path = join(path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [6]:
sum(len(files[i]) for i in range(5))

5000

In [7]:
pathname_list = []
for fo in range(len(categories)):
    for fi in files[fo]:
        pathname_list.append(join(path, join(categories[fo], fi)))

In [8]:
for i in range(len(pathname_list)):
    print(pathname_list[i])

C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52550
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52551
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52552
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52553
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52554
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52555
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52556
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52557
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52558
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52559
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52560
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52561
C:\Users\monac\Downloads\20_newsgroups\20_newsgroups\rec.sport.hockey\52562
C:\Users\mon

In [9]:
Y = []

for folder_name in categories:
    folder_path = join(path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [10]:
len(Y)

5000

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
doc_train, doc_test, Y_train, Y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.50)

In [13]:
l= Y_train
dict(zip(set(l),map(l.count, set(l))))

{'comp.graphics': 513,
 'sci.space': 505,
 'sci.med': 493,
 'rec.sport.hockey': 489,
 'talk.politics.misc': 500}

In [14]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
Y_train= label_encoder.fit_transform(Y_train)
Y_test= label_encoder.fit_transform(Y_test)
 
print(Y_train)

[1 1 0 ... 4 3 3]


In [15]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'its', 'between', 'him', 'they', 'that', 'through', "you've", 'than', 'too', 'herself', 'on', 'should', 'ain', 'now', "hadn't", 'have', 'from', 'at', 'mustn', "you'd", 'couldn', 'theirs', "it's", 'whom', 'same', 'other', 'as', "mustn't", 'doesn', "you're", 'he', 'wasn', 'itself', 'but', 'above', 'when', 'hadn', 'had', 'a', 'any', 'be', 'these', 'my', 'am', 'before', 'few', 'about', 'in', 'did', 'don', 'ma', 'weren', 'those', 'there', 'over', 'of', 'what', 'no', 'who', 'i', 'down', 'd', 's', 'more', 'isn', "doesn't", 'after', 'y', 'during', "shouldn't", 'ourselves', 'do', 'were', 'some', 'themselves', 'not', 'been', 'them', 'was', 'for', "hasn't", 'until', 'under', 'up', 'once', 'while', 'our', 'having', 'm', 'out', 'most', 'because', 't', 've', 'doing', 'his', 'only', 'with', 'below', 'won', 'to', 'here', 'if', 'does', 'then', 'didn', 'your', 'll', "don't", 'yourself', 'is', "didn't", 'wouldn', 'yourselves', "weren't", 'are', 'again', "that'll", 'hasn', "aren't", 'further', 'himself',

In [16]:
len(stop_words)

179

In [17]:
stop_words =  stop_words.union({'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th'})

In [18]:
len(stop_words)

234

In [19]:
def remove_punctuation(text):
    text = " ".join(text)

    text = re.sub(r'[-!$%^&*#()_+|@\]\[~=`{}\\:"\';<>?,.\/]','',text)
    text = re.sub(r'[0-9]+','',text)
    
    text = text.strip()
    return text.split(" ")

def tokenize(text):
    text = word_tokenize(text)
    return text

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

lemmatizer = WordNetLemmatizer()
def lemmatize(text): 
    text = [lemmatizer.lemmatize(token[0]) for token in text]
    return text
 

def remove_whitespace(text):
    temp = list()
    for i in range(len(text)):
        if len(text[i].split()):
            temp.append(text[i].split())
    return temp

def preprocessing(text):
    text = text.lower()
    text = tokenize(text)
    text = remove_stopwords(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = lemmatize(text)
    return text

In [20]:

def remove_metadata(lines):
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines

In [21]:
import joblib
import os
from collections import OrderedDict
#from natsort import natsorted
from pathlib import Path
def tokenize_final(path):
    file = open(path, "r",encoding ="utf8", errors ="surrogateescape")
 
    text_lines = file.read()

    
    
    doc_words = []
    
    
    
    doc_words= (preprocessing(text_lines))
#     print(doc_words)

    return doc_words

In [22]:


def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

In [23]:
len(categories)

5

In [24]:
list_of_words = []

for document in doc_train:
        list_of_words.append(tokenize_final(document))
        

In [25]:
i=0
for document in doc_train:
    print(Y_train[i],list_of_words[i])
    i+=1

1 ['path', 'cantaloupesrvcscmuedu', 'magnesiumclubcccmuedu', 'newsseicmuedu', 'cisohiostateedu', 'zaphodmpsohiostateedu', 'malgudioarnet', 'newsysuedu', 'cmuvmcsvcmichedu', 'cwdtr', 'organization', 'central', 'michigan', 'university', 'date', 'friday', 'apr', 'edt', 'ryan', 'j', 'thieme', 'cwdtr', 'cmuvmcsvcmichedu', 'messageid', 'cwdtr', 'cmuvmcsvcmichedu', 'newsgroups', 'recsporthockey', 'subject', 'thumb', 'espn', 'reference', 'apr', 'wueclwustledu', 'line', 'article', 'apr', 'wueclwustledu', 'jca', 'cecwustledu', 'joseph', 'charles', 'achkar', 'say', 'nice', 'see', 'espn', 'show', 'game', 'wing', 'leaf', 'since', 'cub', 'astros', 'got', 'rained', 'instead', 'showing', 'another', 'baseball', 'game', 'decided', 'stanley', 'cup', 'playoff', 'classy', 'move', 'espn', 'classy', 'move', 'look', 'like', 'espn', 'going', 'devote', 'coverage', 'pen', 'tuesday', 'night', 'continued', 'broadcast', 'pendevil', 'game', 'even', 'though', 'pittsburgh', 'game', 'well', 'control', 'granted', 'show'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
len(list_of_words)

2500

In [27]:
print(i)

2500


In [28]:
len(flatten(list_of_words))

516192

In [29]:

total_words = set(np.asarray(flatten(list_of_words)))

In [30]:
type(stop_words)

set

In [31]:
print(total_words)

{'allright', 'chappaqua', 'motherchild', 'speedaoapower', 'conventionally', 'novastanfordedu', 'koresh', 'simultaneously', 'judgeapproved', 'rhtrinnj', 'opportunitieswhalers', 'radiotv', 'nive', 'trusty', 'shame', 'jmilitzok', 'mouth', 'ohstpy', 'tended', 'labeled', 'pressbox', 'rek', 'superseded', 'komaroff', 'frontends', 'tchecoslovaquie', 'sharpedged', 'wristband', 'sensing', 'incoming', 'chapter', 'hemmorage', 'qmeinnm', 'loc', 'blockd', 'blubird', 'gargle', 'nv', 'rub', 'operates', 'uvietta', 'berlin', 'assult', 'macpc', 'targacapable', 'electrolyte', 'traditionally', 'reload', 'flooding', 'booksthedifferenceengine', 'headlamp', 'tuberous', 'historical', 'shitless', 'spray', 'bask', 'spotty', 'assembler', 'cljgn', 'vides', 'photointelligence', 'fbissov', 'graphing', 'spacefilling', 'dog', 'rstevew', 'linejuggling', 'eindhoven', 'colorresolution', 'stanias', 'tomasjelinek', 'swillbellies', 'wau', 'collaboration', 'curmx', 'maximization', 'carre', 'bowing', 'handpicked', 'malaspina'

In [32]:
len(total_words)

40301

In [33]:
def create_dataframe(list_of_words,total_words):
    df = []
    for i in list_of_words:
        wordDict = dict.fromkeys(total_words,0)
        for word in i:
            wordDict[word]+=1
        df.append(wordDict)
    
    return df
df = create_dataframe(list_of_words,total_words)
    

In [34]:
print(df[0])

{'allright': 0, 'chappaqua': 0, 'motherchild': 0, 'speedaoapower': 0, 'conventionally': 0, 'novastanfordedu': 0, 'koresh': 0, 'simultaneously': 0, 'judgeapproved': 0, 'rhtrinnj': 0, 'opportunitieswhalers': 0, 'radiotv': 0, 'nive': 0, 'trusty': 0, 'shame': 0, 'jmilitzok': 0, 'mouth': 0, 'ohstpy': 0, 'tended': 0, 'labeled': 0, 'pressbox': 0, 'rek': 0, 'superseded': 0, 'komaroff': 0, 'frontends': 0, 'tchecoslovaquie': 0, 'sharpedged': 0, 'wristband': 0, 'sensing': 0, 'incoming': 0, 'chapter': 0, 'hemmorage': 0, 'qmeinnm': 0, 'loc': 0, 'blockd': 0, 'blubird': 0, 'gargle': 0, 'nv': 0, 'rub': 0, 'operates': 0, 'uvietta': 0, 'berlin': 0, 'assult': 0, 'macpc': 0, 'targacapable': 0, 'electrolyte': 0, 'traditionally': 0, 'reload': 0, 'flooding': 0, 'booksthedifferenceengine': 0, 'headlamp': 0, 'tuberous': 0, 'historical': 0, 'shitless': 0, 'spray': 0, 'bask': 0, 'spotty': 0, 'assembler': 0, 'cljgn': 0, 'vides': 0, 'photointelligence': 0, 'fbissov': 0, 'graphing': 0, 'spacefilling': 0, 'dog': 0, 

In [35]:
df = pd.DataFrame(df)

In [36]:
df.loc[0]

allright          0
chappaqua         0
motherchild       0
speedaoapower     0
conventionally    0
                 ..
trearnbitnet      0
kfl               0
cdnswc            0
featuring         0
baxter            0
Name: 0, Length: 40301, dtype: int64

# Term Frequency

In [37]:
def term_frequency(df,Y,total_words):
    tf = pd.DataFrame(columns=total_words)
    categories =[0,1,2,3,4]
    for i in categories:
        
        df_category =pd.DataFrame(columns=total_words)
        for k in range(len(Y)):
            if Y[k] == i:
                df_category.loc[len(df_category.index)] = df.loc[k] 
            
        
        tf.loc[i] = df_category.sum(axis = 0, skipna = True)
        print(tf.loc[i])
    return tf    
                
tf = term_frequency(df,Y_train,total_words)
        
    
            
    
    

allright          0
chappaqua         0
motherchild       0
speedaoapower     0
conventionally    0
                 ..
trearnbitnet      4
kfl               0
cdnswc            2
featuring         2
baxter            1
Name: 0, Length: 40301, dtype: object
allright          0
chappaqua         0
motherchild       0
speedaoapower     0
conventionally    0
                 ..
trearnbitnet      0
kfl               0
cdnswc            0
featuring         2
baxter            0
Name: 1, Length: 40301, dtype: object
allright          0
chappaqua         1
motherchild       0
speedaoapower     0
conventionally    0
                 ..
trearnbitnet      0
kfl               4
cdnswc            0
featuring         0
baxter            0
Name: 2, Length: 40301, dtype: object
allright          0
chappaqua         0
motherchild       1
speedaoapower     1
conventionally    1
                 ..
trearnbitnet      0
kfl               0
cdnswc            0
featuring         2
baxter            0
Name: 

In [38]:
def compute_CF(tf,total_words):
    
    cfDict = {}
    N = len(total_words)
    
    
    cfDict = dict.fromkeys(total_words,0)
    for word in total_words:
        count =0
        for i in range(len(categories)):
            if(tf.iloc[i][word]>0):
                count+=1
        cfDict[word] = count
    return cfDict

cfDict = compute_CF(tf,total_words)
    
    
    
    
    
   

In [39]:
cfDict

{'allright': 1,
 'chappaqua': 1,
 'motherchild': 1,
 'speedaoapower': 1,
 'conventionally': 1,
 'novastanfordedu': 2,
 'koresh': 3,
 'simultaneously': 4,
 'judgeapproved': 1,
 'rhtrinnj': 1,
 'opportunitieswhalers': 1,
 'radiotv': 1,
 'nive': 1,
 'trusty': 1,
 'shame': 4,
 'jmilitzok': 1,
 'mouth': 3,
 'ohstpy': 2,
 'tended': 4,
 'labeled': 3,
 'pressbox': 1,
 'rek': 1,
 'superseded': 1,
 'komaroff': 1,
 'frontends': 1,
 'tchecoslovaquie': 1,
 'sharpedged': 1,
 'wristband': 1,
 'sensing': 2,
 'incoming': 1,
 'chapter': 4,
 'hemmorage': 1,
 'qmeinnm': 1,
 'loc': 1,
 'blockd': 1,
 'blubird': 1,
 'gargle': 1,
 'nv': 1,
 'rub': 2,
 'operates': 4,
 'uvietta': 1,
 'berlin': 2,
 'assult': 1,
 'macpc': 1,
 'targacapable': 1,
 'electrolyte': 1,
 'traditionally': 4,
 'reload': 1,
 'flooding': 1,
 'booksthedifferenceengine': 1,
 'headlamp': 1,
 'tuberous': 1,
 'historical': 3,
 'shitless': 1,
 'spray': 2,
 'bask': 1,
 'spotty': 2,
 'assembler': 1,
 'cljgn': 1,
 'vides': 1,
 'photointelligence': 1

In [40]:
def compute_ICF(cf,total_words):
    
    icfDict = {}
    N = len(total_words)
    
    
    icfDict = dict.fromkeys(total_words,0)
    for word in total_words:
        icfDict[word] = math.log(5/cf[word],2)
    return icfDict

icfDict = compute_ICF(cfDict,total_words)
    

In [41]:
icfDict

{'allright': 2.321928094887362,
 'chappaqua': 2.321928094887362,
 'motherchild': 2.321928094887362,
 'speedaoapower': 2.321928094887362,
 'conventionally': 2.321928094887362,
 'novastanfordedu': 1.3219280948873624,
 'koresh': 0.7369655941662062,
 'simultaneously': 0.32192809488736235,
 'judgeapproved': 2.321928094887362,
 'rhtrinnj': 2.321928094887362,
 'opportunitieswhalers': 2.321928094887362,
 'radiotv': 2.321928094887362,
 'nive': 2.321928094887362,
 'trusty': 2.321928094887362,
 'shame': 0.32192809488736235,
 'jmilitzok': 2.321928094887362,
 'mouth': 0.7369655941662062,
 'ohstpy': 1.3219280948873624,
 'tended': 0.32192809488736235,
 'labeled': 0.7369655941662062,
 'pressbox': 2.321928094887362,
 'rek': 2.321928094887362,
 'superseded': 2.321928094887362,
 'komaroff': 2.321928094887362,
 'frontends': 2.321928094887362,
 'tchecoslovaquie': 2.321928094887362,
 'sharpedged': 2.321928094887362,
 'wristband': 2.321928094887362,
 'sensing': 1.3219280948873624,
 'incoming': 2.321928094887

# Tf-ICF

In [42]:
def computeTFICF(tf,icfDict,total_words):
    tficf = pd.DataFrame(columns=total_words)
    for i in range(len(categories)):
        tficf_dict ={}
        for word in total_words:
            tficf_dict[word]= tf.iloc[i][word]*icfDict[word]
        tficf = tficf.append(tficf_dict, ignore_index = True)
    return tficf
tficf = computeTFICF(tf,icfDict,total_words)

In [43]:
tficf

,allright,chappaqua,motherchild,speedaoapower,conventionally,novastanfordedu,koresh,simultaneously,judgeapproved,rhtrinnj,...,accelerator,itch,drycleaning,liposuction,sphinx,trearnbitnet,kfl,cdnswc,featuring,baxter
0,0.000000,0.000000,0.000000,0.000000,0.000000,5.287712,0.000000,2.253497,0.000000,0.000000,...,3.541209,0.000000,0.000000,0.000000,48.76049,9.287712,0.000000,4.643856,0.643856,2.321928
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.736966,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.643856,0.000000
2,0.000000,2.321928,0.000000,0.000000,0.000000,5.287712,0.736966,0.643856,0.000000,2.321928,...,0.321928,2.321928,0.000000,2.321928,0.00000,0.000000,9.287712,0.000000,0.000000,0.000000
3,0.000000,0.000000,2.321928,2.321928,2.321928,0.000000,0.000000,1.931569,0.000000,0.000000,...,3.863137,0.000000,4.643856,0.000000,0.00000,0.000000,0.000000,0.000000,0.643856,0.000000
4,2.321928,0.000000,0.000000,0.000000,0.000000,0.000000,64.116007,0.965784,2.321928,0.000000,...,0.321928,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.643856,0.000000


# top k features

In [44]:
k=10
features = []
for i in range(0,5):
    index_list= sorted(range(len(tficf.loc[i])), key = lambda sub: tficf.loc[i][sub])[-k:]
    print(len(index_list))
    
    for j in index_list:
        print(tficf.loc[i][j])
        features.append(tficf.columns[j])
features = set(features)

10
148.60339807279118
155.56918235745326
167.17882283189007
181.11039140121426
200.45464161320808
278.63137138648347
325.06993328423073
344.14113343459036
529.8782622055023
1098.2719888817223
10
210.03519433736878
225.22702520407412
260.05594662738457
276.3094432915961
283.2752275762582
441.16633802859883
608.3451608604889
656.9982631590191
680.3249318019971
1225.9780341005273
10
202.00774425520052
211.508495181978
229.87088139384886
245.8786256490494
271.66558710182136
283.2752275762582
339.0015018535549
357.5769266126538
401.6935604155137
901.5549607131811
10
181.11039140121426
183.7480051893434
220.58316901429941
239.1585937733983
274.96104373657136
294.884868050695
301.8506523353571
380.71529132756035
448.1321223132609
549.0393676538237
10
229.87088139384886
257.7340185324972
263.0636908825851
269.343659006934
278.63137138648347
294.884868050695
503.8583965905576
534.0434618240934
543.3311742036427
1236.002768719684


In [45]:
print(features)

{'misclegal', 'socmen', 'orbit', 'recsporthockey', 'gif', 'pt', 'yeast', 'zootorontoedu', 'hiv', 'vitamin', 'infection', 'kelvinjplnasagov', 'talkpoliticsmisc', 'bruin', 'image', 'vga', 'polygon', 'detroit', 'sky', 'prb', 'det', 'tiff', 'nsmca', 'sciastro', 'baalke', 'stephanopoulos', 'spacecraft', 'recfoodcooking', 'dscomsadesyde', 'fbi', 'jpeg', 'playoff', 'cspittedu', 'vertex', 'scimed', 'hockey', 'cramer', 'geb', 'goalie', 'espn', 'candida', 'compgraphics', 'nhl', 'optilinkcom', 'dyer', 'compgraphicsanimation', 'talkreligionmisc', 'quicktime', 'clayton', 'scispace'}


In [46]:
features = sorted(features)

# create test and train data

In [47]:
test_list_of_words = []

for document in doc_test:
        test_list_of_words.append(tokenize_final(document))
        
test_total_words = set(np.asarray(flatten(test_list_of_words)))

def create_test_dataframe(list_of_words,total_words):
    print(list_of_words)
    df = []
    for i in list_of_words:
       
        wordDict = dict.fromkeys(total_words,0)
        for word in i:
            if word in wordDict:
                wordDict[word]+=1
        df.append(wordDict)
    
    return df

test_df = create_test_dataframe(test_list_of_words,features)
test_df = pd.DataFrame(test_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
len(test_df)

2500

In [49]:
features=list(features)

In [50]:
X_train = df.filter(features, axis=1)

In [51]:
X_train.shape

(2500, 50)

In [52]:
test_df.shape

(2500, 50)

In [53]:
X_train = np.asarray(X_train)
test_df =np.asarray(test_df)
Y_train = np.asarray(Y_train)
Y_test = np.asarray(Y_test)

# Naive Bayes

In [54]:
class MultiNB:
    def __init__(self,alpha=1):
        self.alpha = alpha
    
    def _prior(self): 
       
        P = np.zeros((self.n_classes_))
        _, self.dist = np.unique(self.y,return_counts=True)
        for i in self.classes_:
            P[i] = self.dist[i] / self.n_samples
        return P
            
    def fit(self, X, y): 
       
        self.y = y
        self.n_samples, self.n_features = X.shape
        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        self.class_priors_ = self._prior()
        
        # distinct values in each features
        self.uniques = []
        for i in range(self.n_features):
            tmp = np.unique(X[:,i])
            self.uniques.append( tmp )
            
        self.N_yi = np.zeros((self.n_classes_, self.n_features)) # feature count
        self.N_y = np.zeros((self.n_classes_)) # total count 
        for i in range(len(self.classes_)): 
            indices = np.argwhere(self.y==i).flatten()
            columnwise_sum = []
            for j in range(self.n_features):
                columnwise_sum.append(np.sum(X[indices,j]))
            
            self.N_yi[i] = columnwise_sum # 2d
            self.N_y[i] = np.sum(columnwise_sum) # 1d
            
    def _theta(self, x_i, i, h):
     
        
        Nyi = self.N_yi[h,i]
        Ny  = self.N_y[h]
        
        numerator = Nyi + self.alpha
        denominator = Ny + (self.alpha * self.n_features)
        
        return  (numerator / denominator)**x_i
    
    def _likelyhood(self, x, h):
    
        tmp = []
        for i in range(x.shape[0]):
            tmp.append(self._theta(x[i], i,h))
        
        return np.prod(tmp)
    
    def predict(self, X):
        samples, features = X.shape
        self.predict_proba = np.zeros((samples,self.n_classes_))
        
        for i in range(X.shape[0]):
            joint_likelyhood = np.zeros((self.n_classes_))
            
            for h in range(self.n_classes_):
                joint_likelyhood[h]  = self.class_priors_[h] * self._likelyhood(X[i],h) # P(y) P(X|y) 
                
            denominator = np.sum(joint_likelyhood)
            
            for h in range(self.n_classes_):
                numerator = joint_likelyhood[h]
                self.predict_proba[i,h] = (numerator / denominator)
            
        indices = np.argmax(self.predict_proba,axis=1)
        return self.classes_[indices]

In [59]:

nb = MultiNB(alpha=1)
nb.fit(X_train,Y_train)
y_pred = nb.predict(test_df)
me_score =accuracy_score(Y_test,y_pred)

print('Accuracy ',me_score)


Accuracy  0.9948


C:\Users\monac\AppData\Local\Temp/ipykernel_10956/4048030199.py:71: RuntimeWarning: invalid value encountered in double_scalars
  self.predict_proba[i,h] = (numerator / denominator)


In [60]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       487
           1       1.00      1.00      1.00       511
           2       0.99      1.00      0.99       507
           3       0.99      0.99      0.99       495
           4       1.00      1.00      1.00       500

    accuracy                           0.99      2500
   macro avg       0.99      0.99      0.99      2500
weighted avg       0.99      0.99      0.99      2500



In [61]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(Y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[481   0   5   1   0]
 [  0 511   0   0   0]
 [  1   0 505   1   0]
 [  3   0   0 492   0]
 [  0   1   0   1 498]]
